In [ ]:
import numpy as np
import pandas as pd

import os

dataset_path = r'C:\Users\nehaa\OneDrive\Desktop\GrapeDataset.zip'
for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import zipfile
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Paths
zip_file_path = '/content/drive/MyDrive/GrapeDataset.zip'
extract_path = '/content/GrapeDataset'

# Extract dataset
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")
else:
    print(f"Dataset already extracted at {extract_path}")

# Source and target directories
source_data_dir = os.path.join(extract_path, "Grape Plant from Plant Village Dataset")
target_data_dir = '/content/PartialGrapeDataset'

# Verify dataset structure
if os.path.exists(source_data_dir):
    print("Dataset folder structure:")
    print(os.listdir(source_data_dir))  # Should list class folders
else:
    print(f"Error: {source_data_dir} not found.")

# Function to extract partial data
def extract_partial_data(source_dir, target_dir, num_images_per_class):
    os.makedirs(target_dir, exist_ok=True)
    for class_folder in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_folder)
        target_class_path = os.path.join(target_dir, class_folder)
        os.makedirs(target_class_path, exist_ok=True)

        if os.path.isdir(class_path):
            # List only supported image files
            images = [
                img for img in os.listdir(class_path)
                if img.lower().endswith(('.jpg', '.jpeg', '.png'))
            ][:num_images_per_class]

            for image in images:
                src = os.path.join(class_path, image)
                dest = os.path.join(target_class_path, image)
                shutil.copy(src, dest)

    print(f"Data extraction completed. {num_images_per_class} images per class extracted to {target_dir}.")

# Extract partial data
extract_partial_data(source_data_dir, target_data_dir, num_images_per_class=100)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted to /content/GrapeDataset
Dataset folder structure:
['Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy']
Data extraction completed. 100 images per class extracted to /content/PartialGrapeDataset.


In [ ]:
import os
import shutil
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import inception_v3
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import time
from scipy import linalg
from torch.optim.lr_scheduler import ReduceLROnPlateau

class Generator(nn.Module):
    def __init__(self, noise_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 8*8*256),
            nn.BatchNorm1d(8*8*256),
            nn.LeakyReLU(0.2),
            nn.Unflatten(1, (256, 8, 8)),

            nn.ConvTranspose2d(256, 128, 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(128, 64, 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(64, 32, 5, stride=2, padding=2, output_padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),

            nn.ConvTranspose2d(32, 3, 5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
      return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, input_channels=3, input_height=128, input_width=128):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(input_channels, 64, 5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, 5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, 5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Flatten()
        )

        # Calculate conv_out_size dynamically
        with torch.no_grad():
            dummy_input = torch.zeros(1, input_channels, input_height, input_width)
            conv_out = self.main(dummy_input)
            self.conv_out_size = conv_out.view(1, -1).size(1)

        self.fc = nn.Linear(self.conv_out_size, 1)

    def forward(self, x):
        x = self.main(x)
        return self.fc(x.view(x.size(0), -1))



def discriminator_loss(real_output, fake_output):
    real_loss = nn.BCEWithLogitsLoss()(real_output, torch.ones_like(real_output))
    fake_loss = nn.BCEWithLogitsLoss()(fake_output, torch.zeros_like(fake_output))
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return nn.BCEWithLogitsLoss()(fake_output, torch.ones_like(fake_output))


# Function to calculate FID score
def calculate_fid(real_images, fake_images, device, batch_size=64):
    inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.fc = nn.Identity()
    inception_model.eval()

    def get_activations(images):
        activations = []
        with torch.no_grad():
            for i in range(0, images.shape[0], batch_size):
                batch = images[i:i+batch_size]
                batch = torch.nn.functional.interpolate(batch, size=(299, 299), mode='bilinear', align_corners=False)
                batch = batch.to(device)
                pred = inception_model(batch)
                activations.append(pred.cpu().numpy())
        return np.concatenate(activations, axis=0)

    real_activations = get_activations(real_images)
    fake_activations = get_activations(fake_images)

    mu_real, sigma_real = real_activations.mean(axis=0), np.cov(real_activations, rowvar=False)
    mu_fake, sigma_fake = fake_activations.mean(axis=0), np.cov(fake_activations, rowvar=False)

    diff = mu_real - mu_fake
    covmean, _ = linalg.sqrtm(sigma_real.dot(sigma_fake), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma_real + sigma_fake - 2 * covmean)
    return fid


def save_generated_images(epoch, generator, noise_dim, device, output_dir="GrapeDataset/Grape Plant from Plant Village Dataset"):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(100, noise_dim, device=device)  # Generate 100 images
        generated_images = generator(noise).cpu()
        generated_images = (generated_images + 1) / 2  # Scale to [0, 1]

        # Save each generated image in the corresponding class folder
        for i, img_tensor in enumerate(generated_images):
            class_folder = os.path.join(output_dir, f"class_{i % 4}")
            os.makedirs(class_folder, exist_ok=True)
            img = transforms.ToPILImage()(img_tensor)
            img.save(os.path.join(class_folder, f"epoch_{epoch}_image_{i + 1}.png"))

    print(f"Generated images for epoch {epoch} saved to {output_dir}.")


# Function to load image dataset
def load_image_dataset(directory, batch_size):
    transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Ensure dimensions match Discriminator input
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1, 1]
    ])

    dataset = datasets.ImageFolder(directory, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    return dataloader


# Early stopping mechanism based on FID
def should_early_stop(fid_scores, patience=10, threshold=0.1):
    if len(fid_scores) < patience:
        return False
    recent_scores = fid_scores[-patience:]
    return max(recent_scores) - min(recent_scores) < threshold


# Training step
def train_step(images, generator, discriminator, generator_optimizer, discriminator_optimizer, noise_dim):
    batch_size = images.size(0)
    noise = torch.randn(batch_size, noise_dim, device=images.device)

    # Train Generator
    generator_optimizer.zero_grad()
    generated_images = generator(noise)
    fake_output = discriminator(generated_images)
    gen_loss = generator_loss(fake_output)
    gen_loss.backward()
    generator_optimizer.step()

    # Train Discriminator
    discriminator_optimizer.zero_grad()
    real_output = discriminator(images)
    fake_output = discriminator(generated_images.detach())
    disc_loss = discriminator_loss(real_output, fake_output)
    disc_loss.backward()
    discriminator_optimizer.step()

    return gen_loss.item(), disc_loss.item()


def train(dataloader, epochs, noise_dim, generator, discriminator, generator_optimizer, discriminator_optimizer, gen_scheduler, disc_scheduler, device):
    seed = torch.randn(16, noise_dim, device=device)
    fid_scores = []

    start_time = time.time()

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        with tqdm(total=len(dataloader), desc=f"Epoch {epoch + 1}/{epochs}", unit="batch") as pbar:
            for image_batch, _ in dataloader:
                image_batch = image_batch.to(device)
                gen_loss, disc_loss = train_step(image_batch, generator, discriminator, generator_optimizer, discriminator_optimizer, noise_dim)
                pbar.update(1)
                pbar.set_postfix({"Gen Loss": f"{gen_loss:.4f}", "Disc Loss": f"{disc_loss:.4f}"})

        # Save generated images and calculate FID every 5 epochs
        if (epoch + 1) % 5 == 0:
            save_generated_images(epoch, generator=generator, noise_dim=noise_dim, device=device)

            # Add FID calculation here (using actual real_images and fake_images)
            # Example:
            # real_images = get_real_images(dataloader)  # Define function to fetch real images
            # fake_images = get_fake_images(generator, noise_dim, device)
            # fid_score = calculate_fid(real_images, fake_images, device)
            # fid_scores.append(fid_score)
            fid_score = 0  # Placeholder until real FID calculation is implemented

            # Step the schedulers
            gen_scheduler.step(fid_score)
            disc_scheduler.step(fid_score)

        elapsed_time = time.time() - start_time
        estimated_time = (elapsed_time / (epoch + 1)) * epochs
        remaining_time = estimated_time - elapsed_time

        print(f"Estimated time remaining: {remaining_time/60:.2f} minutes")
        print(f"Current learning rates - Generator: {gen_scheduler.optimizer.param_groups[0]['lr']:.6f}, Discriminator: {disc_scheduler.optimizer.param_groups[0]['lr']:.6f}")

    # Plot FID scores if available
    if fid_scores:
        plt.figure(figsize=(10, 5))
        plt.plot(range(5, len(fid_scores) * 5 + 1, 5), fid_scores)
        plt.xlabel('Epoch')
        plt.ylabel('FID Score')
        plt.title('FID Score over Training')
        plt.savefig('fid_scores.png')
        plt.close()



# Main function
def main():
    # Hyperparameters
    BATCH_SIZE = 64
    EPOCHS = 1000
    noise_dim = 100
    initial_lr = 1e-4

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset paths
    source_data_dir = "/content/GrapeDataset/Grape Plant from Plant Village Dataset"
    target_data_dir = "/content/PartialGrapeDataset"
    os.makedirs(target_data_dir, exist_ok=True)

    extract_partial_data(source_data_dir, target_data_dir, num_images_per_class=100)

    # Load dataset
    dataloader = load_image_dataset(target_data_dir, BATCH_SIZE)

    # Model initialization
    generator = Generator(noise_dim).to(device)
    discriminator = Discriminator(3, 128, 128).to(device)

    # Optimizers
    generator_optimizer = optim.Adam(generator.parameters(), lr=initial_lr, betas=(0.5, 0.999))
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=initial_lr, betas=(0.5, 0.999))

    # Learning rate schedulers
    gen_scheduler = ReduceLROnPlateau(generator_optimizer, 'min', factor=0.5, patience=10, min_lr=1e-6)
    disc_scheduler = ReduceLROnPlateau(discriminator_optimizer, 'min', factor=0.5, patience=10, min_lr=1e-6)

    # Training
    train(dataloader, EPOCHS, noise_dim, generator, discriminator, generator_optimizer, discriminator_optimizer, gen_scheduler, disc_scheduler, device)


if __name__ == "__main__":
    main()

Data extraction completed. 100 images per class extracted to /content/PartialGrapeDataset.
Epoch 1/1000


Epoch 1/1000: 100%|██████████| 13/13 [00:04<00:00,  3.22batch/s, Gen Loss=3.4142, Disc Loss=0.0913]


Estimated time remaining: 67.29 minutes
Current learning rates - Generator: 0.000100, Discriminator: 0.000100
Epoch 2/1000


Epoch 2/1000: 100%|██████████| 13/13 [00:05<00:00,  2.43batch/s, Gen Loss=2.1099, Disc Loss=0.2749]


Estimated time remaining: 78.19 minutes
Current learning rates - Generator: 0.000100, Discriminator: 0.000100
Epoch 3/1000


...

Estimated time remaining: 0.16 minutes
Current learning rates - Generator: 0.000001, Discriminator: 0.000001
Epoch 999/1000


Epoch 999/1000: 100%|██████████| 13/13 [00:04<00:00,  2.95batch/s, Gen Loss=3.8170, Disc Loss=0.0352]


Estimated time remaining: 0.08 minutes
Current learning rates - Generator: 0.000001, Discriminator: 0.000001
Epoch 1000/1000


Epoch 1000/1000: 100%|██████████| 13/13 [00:04<00:00,  2.67batch/s, Gen Loss=4.2340, Disc Loss=0.0655]


Generated images for epoch 999 saved to GrapeDataset/Grape Plant from Plant Village Dataset.
Estimated time remaining: 0.00 minutes
Current learning rates - Generator: 0.000001, Discriminator: 0.000001


In [ ]:
import shutil

# Replace `folder_path` with the path to your GAN-generated images folder
folder_path = "/content/GrapeDataset/Grape Plant from Plant Village Dataset"
output_zip = "/content/GrapeDataset.zip"
shutil.make_archive(output_zip.replace(".zip", ""), 'zip', folder_path)


'/content/GrapeDataset.zip'

In [ ]:
from google.colab import files
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>